In [48]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict

In [53]:
with open("varia.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    soup = bs(content, "xml")
with open("navneregister.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    registry = bs(content, "xml")

In [41]:
d = defaultdict(dict)

docDates,regDates,mscontentscount,biblcount = 0,0,0,0
i=1

for item in soup.findAll("TEI"):
    itemDate = "N/A"
    itemType = item['subtype']
    if itemType == "dedication" or itemType == "greeting":
        xmlid = item['xml:id']
        #print(item["xml:id"])
        try:
            bibl = item.find("bibl")
            title = bibl.find("title")
            #print("Title in bibl element")
            biblcount += 1
        except:
            bibl = item.find("msContents")
            title = bibl.find("title")
            #print("Title in msContents element")
            mscontentscount += 1
        for match in title.findAll():
            match.unwrap()
        newTitle = ""
        for x in title.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
            newTitle+=x
        title = newTitle
        itemDate = item.find("docDate")
        try:
            xmldate = itemDate['when']
            docDates+=1

        except:
            itemDate = item.find("date")
            xmldate = itemDate['when']
            regDates+=1
        date = itemDate['when']
        datenumbers = xmldate.replace("-","")
        
        if "Vudat" in xmlid:
            persid = xmlid.replace("Vudat","")
        else:
            persid = xmlid[1:]
        persid = persid.replace(datenumbers,"")
        if "-" in persid: # Indicates date range in xmlid.
            persid = persid[3:] # Remove -XX left behind.
        print("\n")
        d[xmlid]['title'] = title
        print(xmlid,title,itemType,date,", pe/orgID:",persid)
        d[xmlid]['type'] = itemType
        d[xmlid]['date'] = date
        d[xmlid]['ID'] = persid
        i+=1
print(i,"docDate elements:",docDates,"\nregular date elements:",regDates,"\nmscontentscount:",mscontentscount,"\nbibl count:",biblcount)



V18500415CLD Hilsen i Christopher L. Dues stambok greeting 1850-04-15 , pe/orgID: CLD


V18560226NN Dedikasjon til ukjent mottaker i Gildet paa Solhoug dedication 1856-02-26 , pe/orgID: NN


V1856HJ Dedikasjon til Herman Johnson i Gildet paa Solhoug dedication 1856 , pe/orgID: HJ


V18571004MB Dedikasjon til Michael Birkeland i Fru
                     Inger til Østeraad dedication 1857-10-04 , pe/orgID: MB


V185712JCG Dedikasjon til J. Chr. Gamborg i Fru Inger til Østeraad dedication 1857-12 , pe/orgID: JCG


V1858kongO2 Dedikasjon til prins Oscar (senere Oscar II) i Hærmændene paa Helgeland dedication 1858 , pe/orgID: kongO2


V1858CRU Dedikasjon til Carl Richard Unger i Hærmændene paa Helgeland dedication 1858 , pe/orgID: CRU


V1858KK Dedikasjon til Knud Knudsen i Hærmændene paa Helgeland dedication 1858 , pe/orgID: KK


V1858CHo Dedikasjon til Chr. Holst i Hærmændene paa
 Helgeland dedication 1858 , pe/orgID: CHo


V18600819ABla Dedikasjon til August Blanche i Hærmændene
 paa H

In [45]:
iP=0
iO=0
found_persons = []
for item in d:
    whoid = d[item]['ID']
    persid = "pe"+whoid
    orgid = "org"+whoid
    try:
        persdiv = registry.find('div', {'xml:id':persid})
        print("PERS",d[item]['title'],persdiv.find('persName'))
        iP+=1
        found_persons.append(item)
    except:
        try:
            orgid = registry.find('div', {'xml:id':orgid})
            print("ORG",d[item]['title'],orgid.find('item',{'rend':'name'}))
            iO+=1
            found_persons.append(item)
        except:
            pass
print(iP,"personal names\n",iO,"organizations")

PERS Hilsen i Christopher L. Dues stambok <persName><forename>CHRISTOPHER [LORENTZ]</forename><surname>DUE</surname></persName>
PERS Dedikasjon til Michael Birkeland i Fru
                     Inger til Østeraad <persName><forename>MICHAEL</forename><surname>BIRKELAND</surname></persName>
PERS Dedikasjon til J. Chr. Gamborg i Fru Inger til Østeraad <persName><forename>JOACHIM [CHRISTIAN]</forename><surname>GAMBORG</surname>
</persName>
PERS Dedikasjon til Carl Richard Unger i Hærmændene paa Helgeland <persName><forename>CARL RICHARD</forename><surname>UNGER</surname></persName>
PERS Dedikasjon til Knud Knudsen i Hærmændene paa Helgeland <persName><forename>KITTY (Christine Lange)</forename><surname>KIELLAND</surname></persName>
PERS Dedikasjon til Chr. Holst i Hærmændene paa
 Helgeland <persName><forename>CHRISTIAN</forename><surname>HOLST</surname></persName>
PERS Dedikasjon til August Blanche i Hærmændene
 paa Helgeland <persName><forename>AUGUST [THEODOR]</forename><surname>BLANCHE<

PERS Hilsen på tysk skrevet bakpå menykort til Josef
 Lewinsky <persName><forename>JOSEF</forename><surname>LEWINSKY</surname></persName>
PERS Hilsen i Hedvig Stouslands minnebok <persName><forename>HEDVIG (ved dåpen Hedevig Cathrine)</forename><addName>f. Ibsen</addName><surname>STOUSLAND</surname></persName>
PERS Hilsen til Wilhelmine Schønberg på fotografi <persName><forename>WILHELMINE</forename><surname>SCHØNBERG</surname></persName>
PERS Hilsen (i Susannas hånd) til Sophie Krohn på baksiden av
 fotografi <persName><forename>SOPHIE</forename><addName>f. Cappelen</addName><surname>KROHN</surname></persName>
PERS Konvolutt med påskrift i konvolutt til Hildur
 Andersen <persName><forename>HILDUR</forename><surname>ANDERSEN</surname></persName>
PERS Konvolutt med påskrift i konvolutt til Hildur Andersen <persName><forename>HILDUR</forename><surname>ANDERSEN</surname></persName>
PERS Dedikasjon til Rikke Holst i Terje
 Vigen <persName><forename>RIKKE (Henrikke)</forename><surname>HOLST

In [57]:
losttitles = []
for item in d:
    if item not in found_persons and d[item]['ID'] != "NN":
        print(item,"not found. Title",d[item]['title'],"ID:",d[item]['ID'])
        losttitles.append(d[item]['title'])

V1856HJ not found. Title Dedikasjon til Herman Johnson i Gildet paa Solhoug ID: HJ
V1858kongO2 not found. Title Dedikasjon til prins Oscar (senere Oscar II) i Hærmændene paa Helgeland ID: kongO2
V1860Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads
 Abonnenter av Paul Botten Hansen ID: Skand
V18610410EZo not found. Title Dedikasjon til Emilie Zogbaum i et Ibsen-verk ID: EZo
V1861Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads
 Abonnenter av Paul Botten Hansen ID: Skand
V1862Skand not found. Title Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads
 Abonnenter av Paul Botten Hansen ID: Skand
V1862HK not found. Title Dedikasjon til Henriette Konow i Kjærlighedens Komedie ID: HK
V18640616NiFi not found. Title Dedikasjon til Niels Fischer i Kongs-Emnerne ID: NiFi
V1864JPLN not found. Title Dedikasjon til J. Peter L. Nielsen i

In [56]:
i=0
for orgid in registry.findAll('div', {'xml:id':re.compile('org.*')}):
    name = orgid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=x
    name = newname
    print(name)
    i+=1
print(i,"organizations")
i=0
for persid in registry.findAll('div', {'xml:id':re.compile('pe.*')}):
    name = persid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=x
    name = newname
    print(name)
    i+=1
print(i,"persons")

GEORG 2.
SILLE (Cecilie Marie)f. GadeJØRGENSEN
AFTENBLADET
AFTENPOSTEN
BANCO NAPOLETANA
BERGENS NATIONALE SCENE
BERGENS THEATERFORENING
BERLINGSKE TIDENDE
BURGTHEATER
CHRISTIANIA BYFOGEDCONTOR
CHRISTIANIA THEATER
CHRISTIANIA-POSTEN
DAGBLADET
DAGS-TELEGRAPHEN
DEN FRISINNEDE STUDENTERFORENING
DEN GYLDENDALSKE BOGHANDEL
DEN NORSKE FORFATTERFORENING
DEN SKANDINAVISKE FORENING I ROMA
DET AKADEMISKE KOLLEGIUM
DET BRITISK-SKANDINAVISKE SELSKAB
DET KONGELIGE NORSKE VIDENSKABERS SELSKAB
DET KONGELIGE THEATER I KØBENHAVN
DET NORSKE STUDENTERSAMFUND
DET NORSKE THEATER I BERGEN
DEUTSCHE GENOSSENSCHAFT DRAMATISCHER AUTOREN UND COMPONISTEN
DEUTSCHES VOLKSTHEATER
DIREKSJONEN FOR DAMPSKIPET DRONNINGEN
DRESDNER BEZIRKS-GERICHTS-AMT
FELIX BLOCH ERBEN
FINANTS- OG TOLDDEPARTEMENTET (FINANSDEPARTEMENTET)
GIORDANO BRUNO-KOMITEEN
HOHLENBERG
HOTEL GRÖBNER
ILLUSTRERET NYHEDSBLAD
JENSEN  CO.
KIRKE- OG UNDERVISNINGSDEPARTEMENTET, eller: DEPARTEMENTET FOR KIRKE- OG UNDERVISNINGSVESENET, gjerne forkortet: KIRKEDEP

PER (Peder)GYNT
eg.: Publius Aelius HadrianusHADRIAN
[JOHAN FREDRIK] WILHELMHAFFNER
[PETER ANTON] ALFREDHAGE
[EDVARD PHILIP] HOTHERHAGE
[FREDRIK] CHRISTIANHAGEMANN
[GEORG] FRANCISHAGERUP
HANSHAGERUP
AGATHAHAGGENMÜLLER
CARL CHRISTIANHALL

CARL [BRUNNELL]HALS
KARL [MARIUS ANTON JOHAN]HALS
PETTER [MARTIN EMIL NILSON]HALS
SIGURDHALS
JENS BRAAGEHALVORSEN
OLAF [CHRISTOPHERSEN]HALVORSEN
GUSTAF [AXEL KNUT]HAMILTON
CHRISTIAN [MONSEN]HAMMER
[ABRAHAM CHRISTIAN] HJALMAR [FRITHJOF]HAMMER

MARTIN [JOHANNES]HAMMERICH
OLE [LARSEN]HAMMERSTAD
[FRÉDÉRIC ADOLPHE] AUGUSTINHAMON
PEDERHANEVOLD
EMMAHANSCH
NATALIEHANSCH
ASA [CATHARINA]f. EkmanHANSEN
CARL [ANDREAS]HANSEN
frøkenHANSEN
GERHARD [HENRIK] ARMAUERHANSEN
HARALD [FREDRIK]HANSEN
JOHAN [DANIEL] IRGENSHANSEN
LARSHANSEN
[HANS CHRISTIAN] PETERHANSEN
PETER EMMANUEL/PJOTR GOTFRIDOVITSJHANSEN
THEOPHILUS [EDVARD]HANSEN
KLAUSHANSSEN
CHRISTIAN RASMUSHANSSON
OLAF [MØRCH]HANSSON

CHRISTOPHERHANSTEEN
THORAHANSTEEN
HARALD 1. HÅRFAGRE
BENJAMINHARRISON
GUSTAV [LUDVIG T

In [52]:
name

<item rend="name"/>